# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

This is a classification type problem - we are trying to identify whether a student will either pass or fail to graduate. This is a discrete outcome scenario, our features are mostly categorical, we are not trying to quantify a relationship between continuous variables. Therefore it is a classification problem. 

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [11]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1]
n_passed = student_data["passed"].value_counts()["yes"]
n_failed = student_data["passed"].value_counts()["no"]
grad_rate = 100.0*n_passed / n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 31
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [12]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [13]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [22]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=.24)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [84]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
#    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
#    print "Done!\nTraining time (secs): {:.3f}".format(end - start)
    return end-start

# TODO: Choose a model, import it and instantiate an object
from sklearn import svm
clf = svm.SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [82]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
#    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
#    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes'), end-start

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

F1 score for training set: (0.78241758241758241, 0.004571199417114258)


In [51]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.005
F1 score for test set: 0.851612903226


In [103]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
#    print "------------------------------------------"
#    print "Training set size: {}".format(len(X_train))
    time_train = train_classifier(clf, X_train, y_train)
    f1_train, dump = predict_labels(clf, X_train, y_train)
    f1_test, time_test = predict_labels(clf, X_test, y_test)
#    print "F1 score for training set: {}".format(f1_train)
#    print "F1 score for test set: {}".format(f1_test)
    return (len(X_train), time_train, time_test, f1_train, f1_test)

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

from sklearn import svm, tree, ensemble
clf1 = svm.SVC()
clf2 = tree.DecisionTreeClassifier()
clf3 = ensemble.AdaBoostClassifier()
results = []
for clf in [clf1,clf2,clf3]:
    for train_size in [100,200,300]:
        Xt = X_train.sample(train_size)
        yt = y_train.loc[Xt.index.tolist()]
        results.append(train_predict(clf, Xt, yt, X_test, y_test))

line0 = "\t\t\tRBF SVM\t\t\tDecisionTree\t\tAdaBoost"
line1 = "Training set size\t"
line2 = "Training time (sec)\t"
line3 = "Prediction time (sec)\t"
line4 = "F1 training score\t"
line5 = "F1 test score\t\t"
for result in results:
    line1 += str(result[0]) + '\t'
    line2 += "%.5f" % result[1] + '\t'
    line3 += "%.5f" % result[2] + '\t'
    line4 += "%.3f" % result[3] + '\t'
    line5 += "%.3f" % result[4] + '\t'

print line0
print line1
print line2
print line3
print line4
print line5

			RBF SVM			DecisionTree		AdaBoost
Training set size	100	200	300	100	200	300	100	200	300	
Training time (sec)	0.00234	0.00558	0.01038	0.00132	0.00156	0.00204	0.09384	0.08996	0.09351	
Prediction time (sec)	0.00147	0.00260	0.00242	0.00036	0.00020	0.00019	0.00517	0.00511	0.00566	
F1 training score	0.885	0.851	0.870	1.000	1.000	1.000	1.000	0.885	0.856	
F1 test score		0.745	0.831	0.852	0.750	0.763	0.709	0.833	0.750	0.788	


In [ ]:
# TODO: Train and predict using two other models

SVM has complexity O(n*f), its main advantage is the correction factor C, which allows us to avoid overfitting problems by choosing better tolerance for outliers. SVM should have an easier time dealing with the binary data we have created by splitting the data into Boolean features.

Decision trees have complexity O(n), its main advantage is that it can produce simple and easy to understand rules that govern student graduation rates. It is also fast and easy to run. However they are susceptible to overfitting, that is they can fit the training data well, but less so with the test data.

AdaBoost ensembles have complexity O(n*f^2), where f is the number of features. AdaBoost can synthesize many weak learners obeying simple decision rules into a more powerful and accurate high-level classifier. However their execution time increases with the variety of features we have produced, especially by splitting the data into Boolean features.

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?

The most appropriate model for predicting student graduation is the SVM. It does not have the maximal performance on training data (DecisionTree is faster and more accurate on training data), but it does have the best score at handling unseen data (the F1 score on test set). We will want an algorithm with cheap training time and whose score is not very sensitive to different data set sizes (supposing we have a limited amount of data).

- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).

The support vector machine (SVM) is a machine learning algorithm that attempts to find the ideal 'dividing line' that separates passing students from failing students. This algorithm operates over our feature space, which is the full space of possibilities described by all possible combinations of student features.

We use training data - students described by a set of features, along with their known graduation outcomes, to generate such a dividing line. We then use test data - students the algorithm has not seen, along with their known graduation outcomes - to evaluate how well it predicts if those given students passes or fails to graduate. The SVM generates a prediction simply by observing which side of the dividing line a test point falls.

There is also a correction factor built into this algorithm, that makes it more or less tolerant of outliers, or students who don't fit the general trends. A less tolerant machine will find a more complex shape to divide all training points with maximal accuracy, while a more tolerant SVM will choose a simpler shape at the expense of incorrectly classifying some points. This SVM has C = 1, which is more tolerant of outliers.


- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

F1_final = 0.852

In [114]:
# TODO: Fine-tune your model and report the best F1 score
clf = clf1 # chose SVM
from sklearn import grid_search
# grid search via F1 score
# http://stackoverflow.com/questions/34221712/grid-search-with-f1-as-scoring-function-several-pages-of-error-message
from sklearn.metrics import make_scorer
f1x = make_scorer(f1_score, pos_label="yes")
GSCV = grid_search.GridSearchCV(clf,{'C':[0.5,0.75,1,2,3],'kernel':['linear','rbf','poly']},scoring=f1x)
train_predict(GSCV,X_train,y_train,X_test,y_test)
print GSCV.best_params_
print GSCV.best_score_
print f1_score(y_test, clf.predict(X_test), pos_label='yes')

{'kernel': 'rbf', 'C': 1}
0.793585103141
0.851612903226


In [ ]:
Not sure why grid search best f1 score does not match f1_score